In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential,load_model,model_from_json
from tensorflow.keras.layers import Flatten,Dense,Dropout,Convolution2D,MaxPooling2D,BatchNormalization
from sklearn.model_selection import train_test_split 
from tensorflow.keras import utils
from tensorflow.keras.callbacks import LearningRateScheduler

In [4]:
data =pd.read_csv('/content/drive/My Drive/fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
data['Usage'].unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [6]:
train_x,train_y,test_x,test_y=[],[],[],[]
for i,j in data.iterrows():
  if j['Usage']=='Training' or j['Usage']=='PublicTest':
    train_x.append(np.array(j['pixels'].split(),dtype=float))
    train_y.append(np.array(j['emotion'],dtype=int))
  else:
    test_x.append(np.array(j['pixels'].split(),dtype=float))
    test_y.append(np.array(j['emotion'],dtype=int))

In [7]:
train_x,train_y=np.array(train_x).reshape(len(train_x),48,48,1),np.array(train_y)
test_x,test_y=np.array(test_x).reshape(len(test_x),48,48,1),np.array(test_y)

In [8]:
train_y=utils.to_categorical(train_y,len(data['emotion'].unique()))

In [9]:
test_y=utils.to_categorical(test_y,len(data['emotion'].unique()))

In [10]:
train_x.shape

(32298, 48, 48, 1)

In [11]:
train_y.shape

(32298, 7)

In [12]:
test_x.shape

(3589, 48, 48, 1)

In [13]:
test_y.shape

(3589, 7)

In [21]:
model = Sequential()
model.add(Convolution2D(32,(3,3),padding='same',input_shape=(48,48,1),activation='relu',))
model.add(BatchNormalization())
model.add(Convolution2D(32,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(64,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7,activation='softmax'))

In [22]:
def decay_rate(epoch,lr):
  if epoch>50:
    decay_constant=0.001
    new_lr=lr*(1/(1+decay_constant*(epoch-50)))
  else:
    new_lr=lr
  print(f'Previous lr was {lr} and new lr is {new_lr}')
  return new_lr

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=200,batch_size=64,callbacks=[LearningRateScheduler(decay_rate)])

Previous lr was 0.0010000000474974513 and new lr is 0.0010000000474974513
Epoch 1/200
505/505 [==============================] - 11s 21ms/step - loss: 2.1468 - accuracy: 0.2109 - val_loss: 1.7598 - val_accuracy: 0.2739
Previous lr was 0.0010000000474974513 and new lr is 0.0010000000474974513
Epoch 2/200
505/505 [==============================] - 10s 20ms/step - loss: 1.7553 - accuracy: 0.2984 - val_loss: 1.5746 - val_accuracy: 0.3909
Previous lr was 0.0010000000474974513 and new lr is 0.0010000000474974513
Epoch 3/200
505/505 [==============================] - 10s 20ms/step - loss: 1.4800 - accuracy: 0.4268 - val_loss: 1.4178 - val_accuracy: 0.4600
Previous lr was 0.0010000000474974513 and new lr is 0.0010000000474974513
Epoch 4/200
505/505 [==============================] - 10s 20ms/step - loss: 1.3434 - accuracy: 0.4880 - val_loss: 1.2377 - val_accuracy: 0.5230
Previous lr was 0.0010000000474974513 and new lr is 0.0010000000474974513
Epoch 5/200
505/505 [=============================

In [26]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
batch_normalization_14 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
batch_normalization_15 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 64)       

In [27]:
l=model.to_json()
with open('emotion recognition.json','w') as p:
  p.write(l)
model.save_weights('emotion recognition.h5')

In [29]:
model.save('/content/drive/My Drive/emotions.h5')

In [19]:
model=load_model('/content/drive/My Drive/emotions')

In [ ]:
model.predict(test_x[])